# Video Actor Synchroncy and Causality (VASC)
## RAEng: Measuring Responsive Caregiving Project
### Caspar Addyman, 2020
### https://github.com/infantlab/VASC

# Step 3: Analyse the data using scipy

This script uses output from [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) human figure recognition neural network to create labeled wireframes for each figure in each frame of a video. It uses the Python API version of OpenPose.



https://towardsdatascience.com/granger-causality-and-vector-auto-regressive-model-for-time-series-forecasting-3226a64889a6

https://www.machinelearningplus.com/time-series/time-series-analysis-python/

In [1]:
import sys
import os
import json
import numpy as np       #tools for numerical data
import pandas as pd      #data tables & time series
import matplotlib.pyplot as plt # Plotting package

import logging
import ipywidgets as widgets  #let's us add buttons and sliders to this page.
from ipycanvas import Canvas

import matplotlib.pyplot as plt
%matplotlib inline

import vasc #a module of our own functions (found in vasc.py in this folder)

#turn on debugging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
%pdb on   


Automatic pdb calling has been turned ON


In [2]:
# where's the project folder?
jupwd =  os.getcwd() + "\\"
projectpath = os.getcwd() + "\\..\\lookit\\"
#projectpath = os.getcwd() + "\\"

# locations of videos and output
videos_in = projectpath 
videos_out   = projectpath + "out"
#videos_out = "E:\\SpeakNSign\\out"
videos_out_openpose   = videos_out + "\\openpose"
videos_out_timeseries = videos_out + "\\timeseries"
videos_out_analyses   = videos_out + "\\analyses"

print(videos_out_openpose)
print(videos_out_timeseries)
print(videos_out_analyses)

C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\out\openpose
C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\out\timeseries
C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\out\analyses


In [3]:
#retrieve the list of base names of processed videos.
with open(videos_out + '\\clean.json') as json_file:
    videos = json.load(json_file)

In [4]:
#can reload the values without recomputing
reloaded = np.load(videos_out_timeseries + '\\cleandata.npz')
keypoints_original = reloaded["keypoints_array"] #the unprocessed data
keypoints_array = np.copy(keypoints_original)  #an array where we clean the data.

keypoints_array.shape

(10, 3, 412, 4, 75)

## SyncPy Correlation example

It computes the linear correlation between two monovariate signals x and y (in DataFrame format) as a function of their delay tau.
It computes autocorrelation when y coincides with x. 

In [5]:
sys.path.insert(0, '../SyncPy/src/')   # To be able to import packages from parent directory
sys.path.insert(0, '../SyncPy/src/Methods')


In [6]:
print ("\n")
print("************************************************************************************************")
print("This scripts computes the correlation between two monovariate signals."
       "First input is a sinewave of 1 Hz frequency, the second one\n is the sum of this sinewave"
       "with a gaussian random process having zero mean and unitary\n variance.")
print("************************************************************************************************")

""" Import wanted module with every parent packages """
import Methods.DataFrom2Persons.Univariate.Continuous.Linear.Correlation as Correlation

""" Import Utils modules """
from Methods.utils import Standardize
from Methods.utils.ExtractSignal import ExtractSignalFromCSV



************************************************************************************************
This scripts computes the correlation between two monovariate signals.First input is a sinewave of 1 Hz frequency, the second one
 is the sum of this sinewavewith a gaussian random process having zero mean and unitary
 variance.
************************************************************************************************


In [9]:
""" Define signals in pd.dataFrame format """

#Define parameters
N=1024 # number of samples
f=1.0  # sinewave frequency (Hz)
Fs=200 # sampling frequency (Hz)

n=np.arange(0,N)#number of samples

# Create signals
x = pd.DataFrame({'X':np.sin(2*3.14*f*n/Fs)}, np.arange(0,N))
y = pd.DataFrame({'Y':np.sin(2*3.14*f*n/Fs)+10*np.random.randn(1,N)[0]},np.arange(0,N))

'''
"""OR"""
""" Import signals from a .csv file """
#Data from files
filename = 'data_examples/2Persons_Monovariate_Continuous_data.csv'

x = ExtractSignalFromCSV(filename, columns = ['x1'])
y = ExtractSignalFromCSV(filename, columns = ['x2'])
n=np.arange(0,x.shape[0])
'''



"""Plot input signals"""
plt.ion()
f, axarr = plt.subplots(2, sharex=True)
axarr[0].set_title('Input signals')
axarr[0].set_xlabel('Samples')
axarr[1].set_xlabel('Samples')

axarr[0].plot(n, x, label="x")
axarr[1].plot(n, y, label="y", color='r')
axarr[0].legend(loc='best')
axarr[1].legend(loc='best')



""" Define class attributes of the wanted method """

tau_max = 999                       # the maximum lag at which correlation should be computed (in samples)
plot=True                           # plot of the correlation fucntion
standardization = True              # standardization of the time series to mean 0 and variance 1
corr_tau_max = True                 # return of the maximum of correlation and its lag
corr_coeff = True                   # computation of the correlation coefficient (Pearson's version)
scale=True                          # scale factor to have correlaton in [-1,1]

""" Instanciate the class with its attributes """
print("\n")

try : 
    c=Correlation.Correlation(tau_max, plot, standardization, corr_tau_max, corr_coeff, scale)
except TypeError as err :
    print("TypeError in Correlation constructor : \n" + str(err))
    sys.exit(-1)
except ValueError as err :
    print("ValueError in Correlation constructor : \n" + str(err))
    sys.exit(-1)
except Exception as e :
    print("Exception in Correlation constructor : \n" + str(e))
    sys.exit(-1)

print("An instance the class is now created with the following parameters:\n" +
      "tau max = " + str(tau_max) + "\n" +
      "plot = " + str(plot) + "\n" +
      "standardization= " + str(standardization) + "\n" +
      "corr_tau_max = " + str(corr_tau_max) + "\n" +
      "corr_coeff =" + str(corr_coeff) +"\n" +
      "scale =" + str(scale))

""" Compute the method and get the result """
print("\n")
print("Computing...")

try : 
    res= c.compute([x, y])
except TypeError as err :
    print("TypeError in Correlation computation : \n" + str(err))
    sys.exit(-1)
except ValueError as err :
    print("ValueError in Correlation computation : \n" + str(err))
    sys.exit(-1)
except Exception as e :
    print("Exception in Correlation computation : \n" + str(e))
    sys.exit(-1)

""" Display result """
print("\n")
print("**************************************** \n")
print('Correlation complete result :')
print("****************************************\n")
print("Correlation function array:")
print(res['corr_funct'])
print("Maximum value of the correlation %f and lag (in samples) %d:" %(res['max_corr'],res['t_max']))
print("Pearson's correlation coefficient %f:" %(res['corr_coeff']))


plt.close("all")



An instance the class is now created with the following parameters:
tau max = 999
plot = True
standardization= True
corr_tau_max = True
corr_coeff =True
scale =True


Computing...


**************************************** 

Correlation complete result :
****************************************

Correlation function array:
[ 0.00055833  0.00057387  0.00060354 ... -0.00013203 -0.00017167
 -0.00013097]
Maximum value of the correlation 0.012818 and lag (in samples) -383:
Pearson's correlation coefficient 0.066050:
